In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 8.0)
cnt = resample_cnt(cnt, 16.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
# replace lower level features 
# lower level features maybe have a "distribution" over time, or "distribution" over timecourses
# 

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32)

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32)
inputs = [inputs_a, inputs_b]

In [ ]:
fig = plt.figure(figsize=(8,4))
for i_class in range(2):
    ins = var_to_np(inputs[i_class].squeeze())
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

    
plt.title("Spectrum")
plt.xlabel('Frequency [Hz]')

plt.ylabel('Amplitude')
plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
display(fig)
plt.close(fig)

In [ ]:
# mean ... allow to create timecourse to see it
# mean -> gaussian
# demean signal 

In [ ]:
rng = RandomState(394834)
inputs = [rng.randn(160,2**10), rng.randn(160,2**10)]
import resampy

inputs = [resampy.resample(a.T, a.shape[1],a.shape[1] // 2, axis=0).T for a in inputs]
inputs = [resampy.resample(a.T, a.shape[1],a.shape[1] * 2, axis=0).T for a in inputs]
fig = plt.figure(figsize=(8,4))
for i_class in range(2):
    ins = inputs[i_class].squeeze()
    bps = np.abs(np.fft.rfft(ins.squeeze()))
    plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

    
plt.title("Spectrum")
plt.xlabel('Frequency [Hz]')

plt.ylabel('Amplitude')
plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
display(fig)
plt.close(fig)


In [ ]:
fig, axes = plt.subplots(16,20, figsize=(14,14), sharex=True, sharey=True)
for i_class in range(2):
    for i_example in range(len(inputs[i_class])):
        i_row = i_example // 10
        i_col = i_example % 10
        i_col += i_class * 10
        axes[i_row][i_col].plot(inputs[i_class][i_example].squeeze(),
                               color=seaborn.color_palette()[i_class])
fig.suptitle('Input signals')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
labels = ['Right', 'Rest',]
axes[0][-1].legend(lines, labels, bbox_to_anchor=(1,1,0,0))

In [ ]:
from matplotlib.lines import Line2D
plt.figure(figsize=(10,6))
for i_class in range(2):
    plt.plot(inputs[i_class].squeeze().T, color=seaborn.color_palette()[i_class],lw=0.5);
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
plt.legend(lines, ['Right', 'Rest',], bbox_to_anchor=(1,1,0,0))
plt.title('Input signals')